In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

In [1]:
import os

line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
open_api_key = os.environ.get('OPENAI_API_KEY')
port = 5000

In [2]:
from openai import OpenAI

client = OpenAI(api_key=open_api_key)
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=256
        
    )
    return response.choices[0].message.content

In [3]:
print(get_completion("簡介東吳大學"))

東吳大學是一所位於台灣台北市的知名私立大學，創立於1928年，是台灣歷史最悠久的大學之一。學校以培養人文素養和社會責任感為宗旨，致力於提供高品質的教育和研究環境。

東吳大學擁有豐富的學術資源和優秀的師資隊伍，提供多元化的學術課程和研究領域，包括文學、法律、商學、理學、社會科學等。學校注重學生的全面發展，鼓勵學生參與社會服務和實習活動，培養


In [4]:
messages = []

In [6]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            messages.append(prompt)
            prompts = " ".join(messages)
            reply_text = get_completion(prompts)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )
            messages.append(reply_text)
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

        
@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"507999191675699233","quoteToken":"pWH61d8rKo_hwi8lLhdfA0Ii3h9mzWMSsaigIuzTD4z2WQpFC7oRLjymIGhQ8ioRUymks9ToRjrNfJt14USvgSDgTGidTbcdhiZYFXrHQHA4LTs3mAsqoURT2CGqsNHhKxF8REuSH-hGL3FIdNiRIw","text":"你好"},"webhookEventId":"01HXSH15R40WMR32ENY9JTHJMG","deliveryContext":{"isRedelivery":false},"timestamp":1715622417672,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"8ec6252adae149ee9fdea2738a2559d5","mode":"active"}]}


127.0.0.1 - - [14/May/2024 01:46:56] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"507999207429505205","quoteToken":"-Cs-gszBCSeNGpqZXceT3T_HN3qxXntvywQcjvHH7adTOw3Wg2vJmSPREzyfOLRKOaex3sdL5b9xhJnktMTsKKW7oYpww9-WulwXksJs_-AQGfIAftxxFPHjh2knUOgr4Q-C1rAnO4OaSnPbUdYLWA","text":"AI 簡介一下東吳大學"},"webhookEventId":"01HXSH1EVFZV24FBTX5KFEDSZW","deliveryContext":{"isRedelivery":false},"timestamp":1715622427064,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"214f6988f0b54af28aca8db966720ca9","mode":"active"}]}


...

127.0.0.1 - - [14/May/2024 01:48:47] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"507999407078375617","quoteToken":"AsuQHVJJRLeEMXupIIw3p_Ko1bt_PLTYtbbl84_2kIGDfD20t9OWgSlL8dxKjcsAiR30EinfIYpBCPEcDf6O5kG2kquq5SS6jFFLKisbpUaGR-SAgdgJ8DAur8u4sQjjhfVZmTworTp5ujZs9GOJKQ","text":"AI 你還沒回答城區部的地址"},"webhookEventId":"01HXSH5349EEGGYCFNEVJ3XZ4S","deliveryContext":{"isRedelivery":false},"timestamp":1715622546063,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"b1272fbdee624523b50fe95daa12c306","mode":"active"}]}


127.0.0.1 - - [14/May/2024 01:49:08] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"507999433938698385","quoteToken":"71eY4gxBs_-qHSbkZ9rgEHFdn3SsRc69xI7YXx_DBsfWJd0zY6BMdCaMjNlKGoTRtMF7Co4mJjDxnJOxMkTx1Z94lpcocs-5DynJe2uF5JXQO_PAPHYv_TlcXsHXIbrMT0DKBq1Gxyojh8fgk1dCXA","text":"AI 還是不對"},"webhookEventId":"01HXSH5JRM1P9FM77ZTP4P1BMQ","deliveryContext":{"isRedelivery":false},"timestamp":1715622562073,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"a28c141c028a4cff9aeee5144434505b","mode":"active"}]}


127.0.0.1 - - [14/May/2024 01:49:24] "POST / HTTP/1.1" 200 -
